## Dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

!python --version
# !sudo update-alternatives --config python3

%pip install --upgrade DPAD # Install or update to the latest version of DPAD

# Install dependencies only required for this notebook and other development work
%pip install sympy

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


## Data Preprocessing for Gordon Data

In [ ]:
"""
y (neural) data:
    - "theta coherence" has 1 dimension, and "raw vHPC" + "mPFC LFP" have 2 dimensions, for a total shape of (14123335, 3)

z (behavioral) data:
    - "correct" has 1 dimension for a total shape of (14123335, 1)

u (input) data:
    - "show", "delay", "test", and "rest" have 4 dimensions, and "control_input" has 1 dimension, for a total shape of (14123335, 5)
"""

# Load the theta coherence data and behavioral data
theta_data = pd.read_csv('theta_coherence_with_behavior.csv', delim_whitespace=True)

lfp_data = pd.read_csv('filtered_lfp.csv', delim_whitespace=True)
# Combine theta coherence data with LFP data along columns (axis=1)
combined_data = pd.concat([theta_data, lfp_data], axis=1)

# Define the columns for y (neural data), u (control input), and z (behavioral data)
y_columns = ['Theta_coherence', 'vHPC_LFP', 'mPFC_LFP']  # Replace with actual y column names
z_column = ['Correct']  # Replace with the actual behavioral column name for z
u_columns = ["Show", "Delay", "Test", "Rest", "Control_input"]

# Split data into y, u, and z inputs
y_data = combined_data[y_columns]
u_data = combined_data[u_columns]
z_data = combined_data[z_column]

# Perform train-test split (80:20) for just y, u, and z inputs, without targets
yTrain, yTest, uTrain, uTest, zTrain, zTest = train_test_split(
    y_data, u_data, z_data, train_size=0.8, shuffle=False
)

print(yTrain.shape)


## Data Preprocessing for Stanley Data

In [ ]:
from pynwb import NWBHDF5IO

"""
y (neural) data:
    - using LFP1 which has shape dimensions (16749229, 32)
    - LFP2 also exists but documentation does not mention it
    - sampling rate of 2034.5052083 Hz

z (behavioral) data:
    - whisking (0 or 1) has 1 dimension and whisker motion has 1 dimension, for a total shape of (238248, 2)
    - time stamp starts at 5.0358272 s and increases 0.0339968 s, for a sampling rate of ≈ 29.413 Hz

u (opto input) data:
    - u_opto had 1 dimension and u_galvo has 1 dimension, for a total shape of (16749227, 2)

* all three variables have different sizes!
* I can downsample y and u arrays to the length of z (currently commented bellow) but not sure how that affects results
"""

io = NWBHDF5IO('./Stanley_whisk/AB020_1_Proc.nwb', mode="r")
nwbfile = io.read()

y_lfp1 = nwbfile.processing['ecephys'].data_interfaces['LFP1'].electrical_series['ElectricalSeries'].data[:] * float(1000000) # convert to microvolts
#y_lfp2 = nwbfile.processing['ecephys'].data_interfaces['LFP2'].electrical_series['ElectricalSeries'].data[:] * float(1000000) # convert to microvolts 
#y_data = np.concatenate((y_lfp1, y_lfp2), axis = 1)
y_data = y_lfp1

u_opto = nwbfile.acquisition['OptogeneticSeries1'].data[:]
u_opto = np.reshape(u_opto, (u_opto.shape[0], 1))
u_galvo = nwbfile.acquisition['GalvoSeries1'].data[:]
u_galvo = np.reshape(u_galvo, (u_galvo.shape[0], 1))
u_data = np.concatenate((u_opto, u_galvo), axis = 1)

z_whisking = nwbfile.processing['whisker'].data_interfaces['Whisking'].data[:]
print(nwbfile.processing['whisker'].data_interfaces['Whisking'])
print(nwbfile.processing['whisker'].data_interfaces['Whisking'].timestamps[:5])
z_whisking = np.reshape(z_whisking, (z_whisking.shape[0], 1))
z_whiskerMotion = nwbfile.processing['whisker'].data_interfaces['WhiskerMotion'].data[:]
z_whiskerMotion = np.reshape(z_whiskerMotion, (z_whiskerMotion.shape[0], 1))
z_data = np.concatenate((z_whisking, z_whiskerMotion), axis = 1)

print(y_data.shape)
print(z_data.shape)
print(u_data.shape)

#Downsampling to the size of z:
downsampled_y = y_data[::69]
downsampled_u = u_data[::69]

target_size = 238248 #size of z
aligned_y = downsampled_y[:target_size] if downsampled_y.size > target_size else np.pad(downsampled_y, (0, target_size - downsampled_y.size), mode='constant')
aligned_u = downsampled_u[:target_size] if downsampled_u.size > target_size else np.pad(downsampled_u, (0, target_size - downsampled_u.size), mode='constant')
y_data = aligned_y
u_data = aligned_u

yTrain, yTest, uTrain, uTest, zTrain, zTest = train_test_split(
    y_data, u_data, z_data, train_size=0.8, shuffle=False
)

print(yTrain.shape)


## Fitting to DPAD

In [ ]:
from DPAD import DPADModel


idSys = DPADModel()
methodCode = "DPAD_CzNonLin"
args = DPADModel.prepare_args(methodCode)

idSys.fit(Y = yTrain.T, Z = zTrain.T, nx = 6, n1 = 3, **args)


## (ignore for now) Preprocessing Stanley Data in case we want target variables too

In [ ]:
# Load the theta coherence data and behavioral data
theta_data = pd.read_csv('theta_coherence_with_behavior.csv')
lfp_data = pd.read_csv('filtered_lfp.csv')

# Combine theta coherence data with LFP data along columns (axis=1)
combined_data = pd.concat([theta_data, lfp_data], axis=1)

# Define the columns for y (neural data) and z (behavioral data)
y_columns = ['Theta_coherence', 'vHPC_LFP', 'mPFC_LFP'] 
z_column = ['Correct'] 
u_columns = ["Show", "Delay", "Test", "Rest", "Control_input"]

# Create target columns by shifting y and z data by one time step
for col in y_columns:
    combined_data[f'target_{col}'] = combined_data[col].shift(-1)

# Shift the behavioral data by one time step to create target_z
combined_data['target_z'] = combined_data[z_column[0]].shift(-1)

# Drop the last row, as it will have NaNs after shifting
combined_data = combined_data.dropna()

# Split data into y, u, and z inputs
y_data = combined_data[y_columns]
u_data = combined_data[u_columns]
z_data = combined_data[z_column]

# Define targets for y and z separately
y_targets = combined_data[[f'target_{col}' for col in y_columns]]
z_targets = combined_data['target_z']

# Perform train-test split (80:20) while keeping y, u, and z targets separate
y_training, y_testing, u_training, u_testing, z_training, z_testing, target_y_training, target_y_testing, target_z_training, target_z_testing = train_test_split(
    y_data, u_data, z_data, y_targets, z_targets, train_size=0.8, shuffle=False
)

# Convert `target_y_training` and `target_y_testing` to arrays with multiple dimensions
target_y_training = target_y_training.values  # Shape will be (num_train_samples, len(y_columns))
target_y_testing = target_y_testing.values    # Shape will be (num_test_samples, len(y_columns))

# Convert `target_z_training` and `target_z_testing` to 1D arrays
target_z_training = target_z_training.values  # Shape will be (num_train_samples,)
target_z_testing = target_z_testing.values    # Shape will be (num_test_samples,)

# Verify the shapes
print("y_training shape:", y_training.shape)
print("y_testing shape:", y_testing.shape)
print("target_y_training shape:", target_y_training.shape)  # Multi-dimensional
print("target_y_testing shape:", target_y_testing.shape)
print("u_training shape:", u_training.shape)
print("u_testing shape:", u_testing.shape)
print("z_training shape:", z_training.shape)
print("z_testing shape:", z_testing.shape)
print("target_z_training shape:", target_z_training.shape)  # Single-dimensional
print("target_z_testing shape:", target_z_testing.shape)
